In [ ]:
# Uscita trigger dal signal generator: Pulse Sync

%matplotlib widget
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

import h5py
import time
import numpy as np

import sys
sys.path.insert(1, '..\\Instruments\\network\\Signal_Generator')
sys.path.insert(2, '..\\Instruments\\Acquisition_Card')

import RS_SMA100B
from Card_Class import *


ip              = '192.168.40.15'            # Set IP address of the device
Card_Name       = 'PXI1Slot3'
sGen            = RS_SMA100B.SMA100B(ip)
pxie            = PXIe5170R(Card_Name)

SMA100B: Connection successful!
5170R: Available communication!


In [ ]:
# parameters for the Signal Generator
f_min           = 51e6                                  # tryout value
f_max           = 100e6                                  # tryout value
pulse_freq      = np.arange(f_min, f_max, 1e6)          # Set range of frequency of the signal to send to the resonator
amplitude       = -10                           # Set the amplitude of the signal in dBm

pulse_width     = 5                                 # 2 microsec
pulse_delay     = 5
pulse_period    = 1000  


sGen.reset()
sGen.clear()
sGen.pul_gen_params(delay = pulse_delay, width = pulse_width, period = pulse_period)   # da capire quale pulse width
sGen.pul_gen_mode('SING')
sGen.pul_trig_mode('SING')


# parameters for the Acquisition Card
pxie.voltage_range      = 2.0
pxie.coupling           = 'AC' # or DC????
pxie.num_pts            = 1000
pxie.num_records        = 1
pxie.ref_position       = 0
pxie.sample_rate        = 250e6 # Maximum Value: 250.0e6

pxie.open()


# Turn ON the modulation of the Signal Generator
sGen.pul_state(1)
sGen.RF_state(1)

In [ ]:

slip = 0.0      # time of sleep of triggering

fetched_data = []


for i in pulse_freq:
#     #pxie.sample_rate = 2 * i # f_sample > 2*f_max (Nyquist for every freq) 
    
    sGen.RF_freq(i) # guarda come è definito, abbiamo messo la potenza e6   
    for k in amplitude:
        sGen.RF_lvl_ampl(k)
        sGen.pul_state(1)
        sGen.RF_state(1)
        time.sleep(0.1)
        # wfm = pxie.acquisition(sGen.pul_exe_sing_trig,slip)
        wfm = pxie.acquisition(sGen.pul_exe_sing_trig)
        sGen.pul_state(0)
        sGen.RF_state(0)
    #     fetched_data += [{'freq': i, 'data': wfm[0].samples.tolist()}]

    # print('Number of sweeps:', len(fetched_data), '/', len(pulse_freq))


AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
AcquisitionStatus.IN_PROGRESS
Acquisitio

KeyboardInterrupt: 

In [ ]:
def data_amps(y: np.array, toggle_plot = True):
    x = np.arange(len(y))/pxie.sample_rate
    std = np.std(y)
    indices = find_peaks(y, prominence=2*(max(y) - std))
    idx = indices[0]
    offset = 5
    idx = idx[offset:-offset]
    
    if toggle_plot:
        # plt.clf()
        plt.grid()
        plt.plot(x, y)
        plt.scatter(x[idx],y[idx], color = 'orange')
        plt.draw()
    # return {'mean': np.mean(y[idx]), 'std' : np.std(y[idx])}
    return (x, idx)